# 📥 Import Libraries & Dataset

In [248]:
%pip install pandas
%pip install numpy
%pip install scikit-learn==1.5.2
%pip install xgboost
%pip install catboost

## 📦 Libraries

In [249]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb # type: ignore
from sklearn.model_selection import RandomizedSearchCV, train_test_split #type: ignore
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #type: ignore
from catboost import CatBoostRegressor #type: ignore

## 🗃️ Dataset

In [250]:
# Import dataset

url = "https://drive.google.com/uc?id=1wnDUJCke0araT3A7SAk8ZLTIbV00R3ev"
df = pd.read_csv(url)

# 📰 Description

## 📍 Shape

In [251]:
print("Jumlah baris: ", df.shape[0])
print("Jumlah kolom: ", df.shape[1])

Jumlah baris:  8000
Jumlah kolom:  21


## 📍 Structure

In [252]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     8000 non-null   object 
 1   VendorID               8000 non-null   float64
 2   lpep_pickup_datetime   8000 non-null   object 
 3   lpep_dropoff_datetime  8000 non-null   object 
 4   store_and_fwd_flag     8000 non-null   object 
 5   RatecodeID             8000 non-null   float64
 6   PULocationID           8000 non-null   int64  
 7   DOLocationID           8000 non-null   int64  
 8   passenger_count        8000 non-null   float64
 9   trip_distance          8000 non-null   float64
 10  fare_amount            8000 non-null   float64
 11  extra                  8000 non-null   float64
 12  mta_tax                8000 non-null   float64
 13  tip_amount             8000 non-null   float64
 14  tolls_amount           8000 non-null   float64
 15  ehai

## 📍 Columns

In [253]:
df.columns

Index(['ID', 'VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'trip_type', 'congestion_surcharge', 'payment_type'],
      dtype='object')

Berikut ini adalah deskripsi kolom pada dataset:
- **VendorID**: Kode yang menunjukkan penyedia TPEP yang menyediakan catatan.
  - *1 = Creative Mobile Technologies, LLC*
  - *2 = VeriFone Inc*
- **lpep_pickup_datetime**: Tanggal dan waktu ketika meteran dinyalakan.
- **lpep_dropoff_datetime**: Tanggal dan waktu ketika meteran dimatikan.
- **store_and_fwd_flag**: Flag yang menunjukkan apakah catatan perjalanan disimpan dalam memori kendaraan sebelum dikirim ke vendor.
  - *Y = perjalanan yang disimpan dan diteruskan*
  - *N = perjalanan yang tidak disimpan dan diteruskan*
- **RatecodeID**: Kode tarif akhir yang berlaku pada akhir perjalanan.
  - *1 = Tarif standar*
  - *2 = JFK*
  - *3 = Newark*
  - *4 = Nassau atau Westchester*
  - *5 = Tarif negosiasi*
  - *6 = Perjalanan grup*
- **PULocationID**: Zona Taksi TLC di mana meteran dinyalakan.
- **DOLocationID**: Zona Taksi TLC di mana meteran dimatikan.
- **passenger_count**: Jumlah penumpang di kendaraan. Ini adalah nilai yang dimasukkan oleh sopir.
- **trip_distance**: Jarak perjalanan yang telah dilalui dalam mil yang dilaporkan oleh meteran.
- **fare_amount**: Biaya waktu dan jarak yang dihitung oleh meteran.
- **extra**: Biaya tambahan dan surcharge (misalnya, biaya tambahan $0,50 dan $1 untuk jam sibuk dan malam hari).
- **mta_tax**: Pajak MTA sebesar 0,50 dolar yang otomatis dikenakan berdasarkan tarif meteran yang digunakan.
- **tip_amount**: Jumlah tip field ini otomatis terisi untuk tip kartu kredit. Tip tunai tidak termasuk.
- **tolls_amount**: Jumlah total semua tol yang dibayar selama perjalanan.
- **improvement_surcharge**: Biaya perbaikan sebesar $0,30 yang dikenakan pada awal perjalanan. Biaya ini mulai diterapkan pada tahun 2015.
- **total_amount**: Jumlah total yang dibebankan kepada penumpang (tidak termasuk tip tunai).
- **payment_type**: Kode numerik yang menunjukkan bagaimana penumpang membayar perjalanan.
  - *1 = Kartu kredit*
  - *2 = Tunai*
- **trip_type**: Jenis perjalanan.
  - *1 = Inner city*
  - *2 = Outer city*
- **congestion_surcharge**: Jumlah total yang dikumpulkan untuk surcharge kemacetan NYC selama perjalanan.

## 📍 Overview

In [254]:
df.head()

,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,T03315,2.0,2021-07-02 17:19:11,2021-07-02 17:40:02,N,1.0,152,142,1.0,4.30,...,1.0,0.5,5.00,0.00,NaN,0.3,27.55,1.0,2.75,1.0
1,T07720,2.0,2021-07-05 22:48:04,2021-07-05 23:12:02,N,1.0,93,107,2.0,13.52,...,0.5,0.5,12.15,6.55,NaN,0.3,60.75,1.0,2.75,1.0
2,T09695,2.0,2021-07-07 06:29:43,2021-07-07 06:38:42,N,1.0,74,75,1.0,1.55,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
3,T08802,2.0,2021-07-06 15:33:29,2021-07-06 15:43:25,N,1.0,226,129,1.0,1.22,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
4,T01413,2.0,2021-07-01 17:09:07,2021-07-01 18:03:12,N,1.0,55,86,1.0,14.23,...,1.0,0.5,0.00,2.45,NaN,0.3,49.75,1.0,0.00,1.0


In [255]:
df.describe()

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,0.0,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,1.833250,1.176375,98.700125,134.694875,1.270125,3.459532,14.995941,0.330781,0.475875,1.277159,0.351390,NaN,0.296887,18.237415,1.037875,0.599500,1.427625
std,0.372776,0.813541,64.792549,77.750038,0.917885,4.601964,14.543702,0.626111,0.113664,2.476832,1.503395,NaN,0.038558,16.126335,0.190906,1.136344,0.517972
min,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000,-25.000000,-1.000000,-0.500000,-1.140000,0.000000,NaN,-0.300000,-28.550000,1.000000,-2.750000,1.000000
25%,2.000000,1.000000,55.000000,74.000000,1.000000,1.030000,7.000000,0.000000,0.500000,0.000000,0.000000,NaN,0.300000,8.800000,1.000000,0.000000,1.000000
50%,2.000000,1.000000,75.000000,135.000000,1.000000,1.930000,10.200000,0.000000,0.500000,0.000000,0.000000,NaN,0.300000,13.300000,1.000000,0.000000,1.000000
75%,2.000000,1.000000,129.000000,212.000000,1.000000,3.790000,16.500000,0.500000,0.500000,2.040000,0.000000,NaN,0.300000,20.800000,1.000000,0.000000,2.000000
max,2.000000,5.000000,265.000000,265.000000,6.000000,37.600000,156.000000,4.500000,0.500000,47.880000,13.750000,NaN,0.300000,207.480000,2.000000,2.750000,4.000000


# 🛠️ Data Pre-Processing

## 📍 Duplicate Data

In [256]:
# Cek duplikat data
print("Jumlah duplikasi data: ", df.duplicated().sum())

Jumlah duplikasi data:  0


## 📍 Missing Values

In [257]:
# Threshold
MISSING_THRESHOLD = df.shape[0] * 0.5

missing_counts = df.isna().sum().sort_values(ascending=False)
present_counts = df.notna().sum()
missing_percentage = (missing_counts / df.shape[0] * 100).round(2)

missing_data_summary = pd.DataFrame({
    'Missing Values': missing_counts,
    'Present Values': present_counts,
    'Missing Percent': missing_percentage
})

missing_data_summary[missing_data_summary['Missing Values'] > MISSING_THRESHOLD]

,Missing Values,Present Values,Missing Percent
ehail_fee,8000,0,100.0


In [258]:
# Drop ehail_fee
df = df.drop(columns=['ehail_fee'], axis=1)

In [259]:
# Drop payment_type diluar 1, 2
df = df[df['payment_type'].isin([1, 2])]

# 🛠️ Feature Engineering

## 📍 Encoding

In [260]:
# Encoding store_and_fwd_flag
label_encoder = LabelEncoder()

df["store_and_fwd_flag"] = label_encoder.fit_transform(df["store_and_fwd_flag"])

In [261]:
# Convert datetime
df['pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

In [262]:
# Extract time-based features
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_day'] = df['pickup_datetime'].dt.dayofweek
df['pickup_month'] = df['pickup_datetime'].dt.month
df['dropoff_hour'] = df['dropoff_datetime'].dt.hour
df['dropoff_day'] = df['dropoff_datetime'].dt.dayofweek
df['dropoff_month'] = df['dropoff_datetime'].dt.month

In [263]:
# Create trip_duration column
df['trip_duration'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60

In [264]:
def encode_cyclical(df, feature, max_val):
    df[f'{feature}_sin'] = np.sin(2 * np.pi * df_regression[feature] / max_val)
    df[f'{feature}_cos'] = np.cos(2 * np.pi * df_regression[feature] / max_val)

In [265]:
# Cyclinical encoding hour and month
encode_cyclical(df, 'pickup_hour', 24)
encode_cyclical(df, 'dropoff_hour', 24)
encode_cyclical(df, 'pickup_month', 12)
encode_cyclical(df, 'dropoff_month', 12)

KeyError: 'pickup_hour'

In [ ]:
# Drop unnecessary columns
df = df.drop(columns=['pickup_hour', 'pickup_month', 'dropoff_hour', 'dropoff_month', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)

In [ ]:
df.head()

In [ ]:
# One Hot Encoding Day
pickup_day_one_hot = pd.get_dummies(df['pickup_day'], prefix='pickup_day', drop_first=True).astype(int)
dropoff_day_one_hot = pd.get_dummies(df['dropoff_day'], prefix='dropoff_day', drop_first=True).astype(int)

In [ ]:
df = pd.concat([df, pickup_day_one_hot, dropoff_day_one_hot], axis=1)

In [ ]:
# Drop unnecessary columns
df = df.drop(columns=['pickup_day', 'dropoff_day'], axis=1)

In [ ]:
df.head()

# 🛠️ Regresi

## 📍 Training

In [ ]:
df_regression = df.copy()

In [ ]:
# Drop kolom ID
df_regression = df_regression.drop(columns="ID")

In [ ]:
# Identify categorical features
categorical_cols = ['VendorID', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'trip_type', 'payment_type']

In [ ]:
# Convert categorical columns to 'category' dtype
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [ ]:
# Pembagian fitur & target
X = df_regression.drop(columns='trip_distance', axis=1)
y = df_regression['trip_distance']

In [ ]:
# Identify the indices of categorical features
cat_indices = [X.columns.get_loc(col) for col in categorical_cols]

In [ ]:
# Train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define parameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300, 500],
    'depth': [5, 7, 9],
    'subsample': [1.0],
    'colsample_bylevel': [1.0],
    'min_data_in_leaf': [10, 20, 50],
    'l2_leaf_reg': [0.1, 0.2, 0.3],
    'random_strength': [0, 0.1, 0.2],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
    'leaf_estimation_iterations': [1, 5, 10],
    'border_count': [32, 64, 128]
}

# Initialize CatBoostRegressor
model = CatBoostRegressor(
    loss_function='RMSE',
    verbose=0,
    od_wait=20,
    od_pval=0.01,
    use_best_model=True,
    task_type='CPU'
)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=100,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

In [ ]:
random_search.fit(X_train, y_train)


In [ ]:
best_params = random_search.best_params_
print(best_params)

best_model = random_search.best_estimator_
print(best_model)

In [ ]:
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {rmse:.2f}')
print(f'R-squared: {r2:.2f}')

## 📍 Kaggle

In [ ]:
url = "https://drive.google.com/uc?id=192s2gBVRe7_5kTDkoIRM6Tls7SPNsnvf"
df_kaggle_regression = pd.read_csv(url)

In [ ]:
df_kaggle_regression.head()

In [ ]:
df_kaggle_regression = df_kaggle_regression.drop(columns=['Unnamed: 0', 'ehail_fee'])

In [ ]:
df_kaggle_regression.info()

In [ ]:
processed_df_kaggle_regression = df_kaggle_regression.copy()

In [ ]:
processed_df_kaggle_regression["store_and_fwd_flag"] = label_encoder.transform(processed_df_kaggle_regression["store_and_fwd_flag"])

In [ ]:
processed_df_kaggle_regression

In [ ]:
processed_df_kaggle_regression['pickup_datetime'] = pd.to_datetime(processed_df_kaggle_regression['lpep_pickup_datetime'])
processed_df_kaggle_regression['dropoff_datetime'] = pd.to_datetime(processed_df_kaggle_regression['lpep_dropoff_datetime'])

In [ ]:
processed_df_kaggle_regression['pickup_hour'] = processed_df_kaggle_regression['pickup_datetime'].dt.hour
processed_df_kaggle_regression['pickup_day'] = processed_df_kaggle_regression['pickup_datetime'].dt.dayofweek
processed_df_kaggle_regression['pickup_month'] = processed_df_kaggle_regression['pickup_datetime'].dt.month
processed_df_kaggle_regression['dropoff_hour'] = processed_df_kaggle_regression['dropoff_datetime'].dt.hour
processed_df_kaggle_regression['dropoff_day'] = processed_df_kaggle_regression['dropoff_datetime'].dt.dayofweek
processed_df_kaggle_regression['dropoff_month'] = processed_df_kaggle_regression['dropoff_datetime'].dt.month


In [ ]:
processed_df_kaggle_regression['trip_duration'] = (processed_df_kaggle_regression['dropoff_datetime'] - processed_df_kaggle_regression['pickup_datetime']).dt.total_seconds() / 60

In [ ]:
def encode_test_cyclical(feature, max_val):
    processed_df_kaggle_regression[f'{feature}_sin'] = np.sin(2 * np.pi * processed_df_kaggle_regression[feature] / max_val)
    processed_df_kaggle_regression[f'{feature}_cos'] = np.cos(2 * np.pi * processed_df_kaggle_regression[feature] / max_val)

In [ ]:
encode_test_cyclical('pickup_hour', 24)
encode_test_cyclical('dropoff_hour', 24)
encode_test_cyclical('pickup_month', 12)
encode_test_cyclical('dropoff_month', 12)

In [ ]:
processed_df_kaggle_regression = processed_df_kaggle_regression.drop(columns=['pickup_hour', 'pickup_month', 'dropoff_hour', 'dropoff_month', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)

In [ ]:
test_pickup_day_one_hot = pd.get_dummies(processed_df_kaggle_regression['pickup_day'], prefix='pickup_day', drop_first=True).astype(int)
test_dropoff_day_one_hot = pd.get_dummies(processed_df_kaggle_regression['dropoff_day'], prefix='dropoff_day', drop_first=True).astype(int)

In [ ]:
processed_df_kaggle_regression = pd.concat([processed_df_kaggle_regression, test_pickup_day_one_hot, test_dropoff_day_one_hot], axis=1)

In [ ]:
processed_df_kaggle_regression = processed_df_kaggle_regression.drop(columns=['pickup_day', 'dropoff_day'], axis=1)

In [ ]:
processed_df_kaggle_regression.head()

In [ ]:
processed_df_kaggle_regression = processed_df_kaggle_regression.drop(columns="ID")

In [ ]:
y_result = best_model.predict(processed_df_kaggle_regression)

In [ ]:
csv_result = pd.DataFrame({
    "ID": df_train_regression["ID"],
    "trip_distance": y_result
})

In [ ]:
csv_result

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

utc_plus_7 = ZoneInfo("Asia/Bangkok")
current_iso_timestamp = datetime.now(utc_plus_7).isoformat()

filename = f"submission_kasdead_{current_iso_timestamp}.csv"
csv_result.to_csv(filename, index=False)